In [10]:
import pandas as pd
import numpy as np

In [11]:
stops = pd.read_csv('gtfs_2019-11-14/stops.txt')
trips = pd.read_csv('gtfs_2019-11-14/trips.txt')
routes = pd.read_csv('gtfs_2019-11-14/routes.txt')
stop_times = pd.read_csv('gtfs_2019-11-14/stop_times.txt')

# Stop Times Simple

In [12]:
trips = trips[trips['service_id'] == 1]

In [13]:
trips.to_csv('gtfs_2019-11-14/trips_weekday.csv')

In [14]:
trips = trips.merge(routes)

In [15]:
stop_times = stop_times.merge(trips)

In [16]:
stop_times['hr'] = (stop_times['arrival_time'].str.split(':').str[0]).astype(int)
stop_times['min'] = (stop_times['arrival_time'].str.split(':').str[1]).astype(int)
stop_times['sec'] = (stop_times['arrival_time'].str.split(':').str[2]).astype(int)
stop_times_simple = stop_times[['trip_id', 'route_short_name','stop_id', 'stop_sequence', 'hr', 'min','sec']]

In [17]:
stop_times_simple['min'] = np.where(stop_times_simple['sec']>29, stop_times_simple['min'] + 1, stop_times_simple['min'])
stop_times_simple = stop_times_simple[['trip_id', 'route_short_name', 'stop_id', 'stop_sequence', 'hr', 'min']]
stop_times_simple

/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_24524/2614556194.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stop_times_simple['min'] = np.where(stop_times_simple['sec']>29, stop_times_simple['min'] + 1, stop_times_simple['min'])


,trip_id,route_short_name,stop_id,stop_sequence,hr,min
0,39152872,10,14155,1,9,0
1,39152872,10,3807,2,9,1
2,39152872,10,6904,3,9,2
3,39152872,10,1163,4,9,4
4,39152872,10,7723,5,9,5
...,...,...,...,...,...,...
1383717,39261874,4,14535,1,26,25
1383718,39261874,4,14536,2,26,27
1383719,39261874,4,14537,3,26,29
1383720,39261874,4,14538,4,26,31


# Stop Times Reduced

In [18]:
int_stop = pd.read_csv('int_stop_19.csv')[['INT_ID', 'stop_id', 'dist']]
int_stop

,INT_ID,stop_id,dist
0,100,641.0,266.743646
1,100,13296.0,200.034951
2,100,9227.0,188.435229
3,100,14451.0,188.435229
4,100,14420.0,188.435229
...,...,...,...
9179,30112870,14880.0,61.822357
9180,30113231,6006.0,23.707970
9181,30113231,10234.0,22.215103
9182,30113634,8305.0,29.855818


In [19]:
stop_times_simple = stop_times_simple.merge(int_stop)
stop_times_simple

,trip_id,route_short_name,stop_id,stop_sequence,hr,min,INT_ID,dist
0,39152872,10,14155,1,9,0,404,36.997541
1,39152874,10,14155,1,6,15,404,36.997541
2,39152876,10,14155,1,5,45,404,36.997541
3,39152877,10,14155,1,17,0,404,36.997541
4,39152878,10,14155,1,16,50,404,36.997541
...,...,...,...,...,...,...,...,...
1246336,39261866,4,14539,5,26,11,114,0.000000
1246337,39261867,4,14539,5,26,17,114,0.000000
1246338,39261868,4,14539,5,26,22,114,0.000000
1246339,39261869,4,14539,5,26,28,114,0.000000


In [20]:
stop_times_reduced = stop_times_simple.dropna(subset =['INT_ID'
                                      ]).sort_values(by = 'dist', ascending = True).drop_duplicates(subset = [
        'trip_id', 'INT_ID'], keep = 'first').sort_values(by = ['trip_id', 'stop_sequence'])

In [21]:
stop_times_reduced

,trip_id,route_short_name,stop_id,stop_sequence,hr,min,INT_ID,dist
27,39152871,10,14155,1,18,30,404,36.997541
181,39152871,10,3807,2,18,31,13446349,12.783509
405,39152871,10,6904,3,18,32,13446105,9.598783
629,39152871,10,1163,4,18,34,13445637,23.229400
947,39152871,10,7723,5,18,35,13445327,19.189726
...,...,...,...,...,...,...,...,...
1245355,39262012,4,14535,1,18,38,404,0.000000
1245584,39262012,4,14536,2,18,40,403,0.000000
1245813,39262012,4,14537,3,18,41,402,0.000000
1246042,39262012,4,14538,4,18,43,401,0.000000


In [22]:
g = stop_times_reduced.groupby('trip_id', as_index=False)
stop_times_reduced['stop_sequence_new'] = g.cumcount() + 1

In [23]:
stop_times_reduced['hr'] = np.where(stop_times_reduced['min'] == 60, 
                                        stop_times_reduced['hr'] + 1, stop_times_reduced['hr'])
stop_times_reduced['min'] = np.where(stop_times_reduced['min'] == 60, 
                                        0, stop_times_reduced['min'])

In [24]:
stop_times_reduced.to_csv('gtfs_2019-11-14/stop_times_reduced.csv', index = False)

In [25]:
stop_times_reduced

,trip_id,route_short_name,stop_id,stop_sequence,hr,min,INT_ID,dist,stop_sequence_new
27,39152871,10,14155,1,18,30,404,36.997541,1
181,39152871,10,3807,2,18,31,13446349,12.783509,2
405,39152871,10,6904,3,18,32,13446105,9.598783,3
629,39152871,10,1163,4,18,34,13445637,23.229400,4
947,39152871,10,7723,5,18,35,13445327,19.189726,5
...,...,...,...,...,...,...,...,...,...
1245355,39262012,4,14535,1,18,38,404,0.000000,1
1245584,39262012,4,14536,2,18,40,403,0.000000,2
1245813,39262012,4,14537,3,18,41,402,0.000000,3
1246042,39262012,4,14538,4,18,43,401,0.000000,4


# Stop Times Full

In [26]:
stop_times_reduced['hr'] = np.where(stop_times_reduced['hr']>23, stop_times_reduced['hr'] - 24, stop_times_reduced['hr'])

stop_times_reduced_o = stop_times_reduced.copy()
stop_times_reduced_d = stop_times_reduced.copy()
stop_times_reduced_d['stop_sequence_new'] = stop_times_reduced_d['stop_sequence_new'] - 1

In [27]:
stop_times_od = stop_times_reduced_o.merge(stop_times_reduced_d, left_on = ['trip_id', 'stop_sequence_new'],
                           right_on = ['trip_id', 'stop_sequence_new'], suffixes = ['_o', '_d'])

In [28]:
stop_times_od['min_o'] = stop_times_od['min_o'].apply(lambda x: '{0:0>2}'.format(x))
stop_times_od['min_d'] = stop_times_od['min_d'].apply(lambda x: '{0:0>2}'.format(x))

In [29]:
stop_times_od['hr_o'] = stop_times_od['hr_o'].astype(str)
stop_times_od['hr_d'] = stop_times_od['hr_d'].astype(str)

In [30]:
stop_times_od['o_time_str'] = stop_times_od['hr_o'] + ':' + stop_times_od['min_o']
stop_times_od['d_time_str'] = stop_times_od['hr_d'] + ':' + stop_times_od['min_d']

In [31]:
stop_times_od['o_time'] = pd.to_datetime(stop_times_od['o_time_str'], format = '%H:%M')
stop_times_od['d_time'] = pd.to_datetime(stop_times_od['d_time_str'], format = '%H:%M')

stop_times_od['cost'] = ((stop_times_od['d_time'] - stop_times_od['o_time']).dt.seconds)/60

stop_times_od['o_time'] = stop_times_od['o_time'].dt.time
stop_times_od['d_time'] = stop_times_od['d_time'].dt.time

In [32]:
stop_times_od = stop_times_od.drop(columns = ['o_time_str', 'd_time_str', 'o_time', 'd_time'])

In [33]:
stop_times_od

,trip_id,route_short_name_o,stop_id_o,stop_sequence_o,hr_o,min_o,INT_ID_o,dist_o,stop_sequence_new,route_short_name_d,stop_id_d,stop_sequence_d,hr_d,min_d,INT_ID_d,dist_d,cost
0,39152871,10,14155,1,18,30,404,36.997541,1,10,3807,2,18,31,13446349,12.783509,1.0
1,39152871,10,3807,2,18,31,13446349,12.783509,2,10,6904,3,18,32,13446105,9.598783,1.0
2,39152871,10,6904,3,18,32,13446105,9.598783,3,10,1163,4,18,34,13445637,23.229400,2.0
3,39152871,10,1163,4,18,34,13445637,23.229400,4,10,7723,5,18,35,13445327,19.189726,1.0
4,39152871,10,7723,5,18,35,13445327,19.189726,5,10,2498,6,18,36,13445096,75.177844,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131119,39262011,4,14538,4,18,38,401,0.000000,4,4,14539,5,18,40,114,0.000000,2.0
1131120,39262012,4,14535,1,18,38,404,0.000000,1,4,14536,2,18,40,403,0.000000,2.0
1131121,39262012,4,14536,2,18,40,403,0.000000,2,4,14537,3,18,41,402,0.000000,1.0
1131122,39262012,4,14537,3,18,41,402,0.000000,3,4,14538,4,18,43,401,0.000000,2.0


In [34]:
stop_times_od.to_csv('gtfs_2019-11-14/stop_times_od.csv', index = False)